In [118]:
import math
import re
import time
import string
import pandas as pd
import numpy as np
import hdbscan
import logging
from umap import UMAP
import networkx as nx
import jellyfish
from segtok.segmenter import split_multi
from segtok.tokenizer import web_tokenizer, split_contractions
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

from fasttext import load_model
# fasttext_model = 'cc.en.300.bin'
# fmodel = load_model(fasttext_model)

In [119]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [202]:
# documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(text)]
# doc2vec_args = {"vector_size": 300,
#                 "min_count": 1,
#                 "window": 15,
#                 "sample": 1e-5,
#                 "negative": 0,
#                 "hs": 1,
#                 "epochs": 50,
#                 "dm": 0,
#                 "dbow_words": 1,
#                 "workers":8,
#                "documents":documents}
# model = Doc2Vec(**doc2vec_args)
vec=model.dv.get_normed_vectors()

In [97]:
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 177.27208542823792 seconds ---


In [206]:
o = getClusers(vec)

In [208]:
len(set(o))

46

In [205]:
def getClusers(embeddings):
    outliers = len(embeddings)*.22
    min_cluster_size, min_samples = 15, 2
    clusters = np.full(embeddings.shape[0], -1)
    _clusters = np.arange(len(clusters))
    umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', low_memory=False, verbose=False)
    if len(embeddings)//50000 < 2:
        _umap_embeddings = umap_model.fit_transform(embeddings)
        _embeddings = np.nan_to_num(_umap_embeddings) 
    else:
        umap_model.fit(embeddings[np.random.choice(embeddings.shape[0], 60000, replace=False), :])
        chunks = np.array_split(embeddings, len(embeddings)//50000)
        _umap_embeddings = []
        for chunk in chunks:
            _umap_embeddings.extend(umap_model.transform(chunk))
        _embeddings = np.nan_to_num(_umap_embeddings)    
    while(len(_clusters)>outliers):
        nbrCl = len(set(clusters))-1
        hdbscan_model =hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean',cluster_selection_method='eom', prediction_data=True)
        hdbscan_model.fit(_embeddings)
        indexes = np.argwhere(hdbscan_model.labels_!=-1)
        for i, v in zip(indexes, hdbscan_model.labels_[indexes]):
            clusters[_clusters[i]] = nbrCl + v
        _clusters = np.argwhere(hdbscan_model.labels_==-1)
        _embeddings = _embeddings[_clusters]
        if _embeddings.size == 0: break
        _embeddings = _embeddings.reshape(len(_clusters),-1)
        min_cluster_size, min_samples = 8, 1
    return clusters

In [197]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', low_memory=False, verbose=False)
_umap_embeddings = umap_model.fit_transform(vec)
vec = np.nan_to_num(_umap_embeddings)

In [198]:
hdbscan_model =hdbscan.HDBSCAN(min_cluster_size=10,metric='euclidean', cluster_selection_method='eom', prediction_data=True)
hdbscan_model.fit(vec)


HDBSCAN(min_cluster_size=10, prediction_data=True)

In [199]:
len(set(hdbscan_model.labels_))

140

In [117]:
pd.DataFrame(hdbscan_model.labels_).value_counts()

 31    15934
-1       591
 22      330
 33      221
 27      118
 18      100
 4        96
 37       94
 26       78
 9        75
 21       74
 2        70
 39       64
 38       63
 36       57
 24       56
 5        54
 0        53
 20       52
 6        51
 35       48
 40       42
 17       36
 10       35
 34       34
 14       33
 13       33
 12       31
 30       31
 16       30
 19       28
 7        28
 25       27
 28       25
 11       22
 8        21
 1        21
 23       19
 32       18
 3        18
 29       18
 15       17
dtype: int64

In [22]:
vecs[0]

array([-4.23317775e-02,  6.58288971e-02,  1.25106825e-02, -4.74178381e-02,
        9.59125534e-02, -8.63540992e-02,  1.06869541e-01,  4.76633608e-02,
       -5.09298705e-02,  1.33005483e-03,  3.01549155e-02, -3.32995206e-02,
       -2.41408721e-01, -4.43766341e-02,  3.22487056e-02,  4.10272405e-02,
        2.75378376e-02, -3.78786437e-02,  7.90869370e-02, -7.90262222e-02,
       -9.73701254e-02, -3.10429204e-02, -1.16537482e-01,  3.34297046e-02,
        7.74220154e-02,  1.29241766e-02, -1.26352319e-02,  4.17493656e-02,
       -8.17062855e-02,  5.59743047e-02,  3.79567780e-02, -2.77244654e-02,
        6.10271618e-02, -1.00215171e-02,  2.29185987e-02,  7.88366199e-02,
       -3.42737697e-03,  6.49926718e-03,  3.69488075e-02, -2.08858456e-02,
        2.59066354e-02, -8.15460235e-02, -2.62043942e-02, -2.34507141e-03,
        1.02848105e-01, -1.46775749e-02,  3.36938538e-02, -1.74646638e-02,
        4.09522206e-02, -5.62588591e-03,  1.36226401e-01, -1.82062000e-01,
       -7.86272157e-03,  

In [120]:
STOPWORD_WEIGHT = 'bi'
class DataCore(object):
    
    def __init__(self, text, stopword_set, windowsSize, n, tagsToDiscard = set(['u', 'd']), exclude = set(string.punctuation)):
        self.number_of_sentences = 0
        self.number_of_words = 0
        self.terms = {}
        self.candidates = {}
        self.sentences_obj = []
        self.sentences_str = []
        self.G = nx.DiGraph()
        self.exclude = exclude
        self.tagsToDiscard = tagsToDiscard
        self.freq_ns = {}
        for i in range(n):
            self.freq_ns[i+1] = 0.
        self.stopword_set = stopword_set
        self._build(text, windowsSize, n)

    # Build the datacore features
    def _build(self, text, windowsSize, n):
        text = self.pre_filter(text)
        self.sentences_str = [ [w for w in split_contractions(web_tokenizer(s)) if not (w.startswith("'") and len(w) > 1) and len(w) > 0] for s in list(split_multi(text)) if len(s.strip()) > 0]
        self.number_of_sentences = len(self.sentences_str)
        pos_text = 0
        block_of_word_obj = []
        sentence_obj_aux = []
        for (sentence_id, sentence) in enumerate(self.sentences_str):
            sentence_obj_aux = []
            block_of_word_obj = []
            for (pos_sent, word) in enumerate(sentence):
                if len([c for c in word if c in self.exclude]) == len(word): # If the word is based on exclude chars
                    if len(block_of_word_obj) > 0:
                        sentence_obj_aux.append( block_of_word_obj )
                        block_of_word_obj = []
                else:
                    tag = self.getTag(word, pos_sent)
                    term_obj = self.getTerm(word)
                    term_obj.addOccur(tag, sentence_id, pos_sent, pos_text)
                    pos_text += 1

                    #Create co-occurrence matrix
                    if tag not in self.tagsToDiscard:
                        word_windows = list(range( max(0, len(block_of_word_obj)-windowsSize), len(block_of_word_obj) ))
                        for w in word_windows:
                            if block_of_word_obj[w][0] not in self.tagsToDiscard: 
                                self.addCooccur(block_of_word_obj[w][2], term_obj)
                    #Generate candidate keyphrase list
                    candidate = [ (tag, word, term_obj) ]
                    cand = composed_word(candidate)
                    self.addOrUpdateComposedWord(cand)
                    word_windows = list(range( max(0, len(block_of_word_obj)-(n-1)), len(block_of_word_obj) ))[::-1]
                    for w in word_windows:
                        candidate.append(block_of_word_obj[w])
                        self.freq_ns[len(candidate)] += 1.
                        cand = composed_word(candidate[::-1])
                        self.addOrUpdateComposedWord(cand)

                    # Add term to the block of words' buffer
                    block_of_word_obj.append( (tag, word, term_obj) )

            if len(block_of_word_obj) > 0:
                sentence_obj_aux.append( block_of_word_obj )

            if len(sentence_obj_aux) > 0:
                self.sentences_obj.append(sentence_obj_aux)

        if len(block_of_word_obj) > 0:
            sentence_obj_aux.append( block_of_word_obj )

        if len(sentence_obj_aux) > 0:
            self.sentences_obj.append(sentence_obj_aux)

        self.number_of_words = pos_text

    def build_single_terms_features(self, features=None):
        validTerms = [ term for term in self.terms.values() if not term.stopword ]
        validTFs = (np.array([ x.tf for x in validTerms ]))

        if len(validTFs) == 0:
            return

        avgTF = validTFs.mean()
        stdTF = validTFs.std()
        maxTF = max([ x.tf for x in self.terms.values()])
        list(map(lambda x: x.updateH(maxTF=maxTF, avgTF=avgTF, stdTF=stdTF, number_of_sentences=self.number_of_sentences, features=features), self.terms.values()))

    def build_mult_terms_features(self, features=None):
        list(map(lambda x: x.updateH(features=features), [cand for cand in self.candidates.values() if cand.isValid()]))

    def pre_filter(self, text):
        prog = re.compile("^(\\s*([A-Z]))")
        parts = text.split('\n')
        buffer = ''
        for part in parts:
            sep = ' '
            if prog.match(part):
                sep = '\n\n'
            buffer += sep + part.replace('\t',' ')
        return buffer

    def getTag(self, word, i):
        try:
            w2 = word.replace(",","")
            float(w2)
            return "d"
        except:
            cdigit = len([c for c in word if c.isdigit()])
            calpha = len([c for c in word if c.isalpha()])
            if ( cdigit > 0 and calpha > 0 ) or (cdigit == 0 and calpha == 0) or len([c for c in word if c in self.exclude]) > 1:
                return "u"
            if len(word) == len([c for c in word if c.isupper()]):
                return "a"
            if len([c for c in word if c.isupper()]) == 1 and len(word) > 1 and word[0].isupper() and i > 0:
                return "n"
        return "p"

    def getTerm(self, str_word, save_non_seen=True):
        unique_term = str_word.lower()
        simples_sto = unique_term in self.stopword_set
        if unique_term.endswith('s') and len(unique_term) > 3:
            unique_term = unique_term[:-1]

        if unique_term in self.terms:
            return self.terms[unique_term]
                
        # Include this part
        simples_unique_term = unique_term
        for pontuation in self.exclude:
            simples_unique_term = simples_unique_term.replace(pontuation, '')
        # until here
        isstopword = simples_sto or unique_term in self.stopword_set or len(simples_unique_term) < 3
        
        term_id = len(self.terms)
        term_obj = single_word(unique_term, term_id, self.G)
        term_obj.stopword = isstopword

        if save_non_seen:
            self.G.add_node(term_id)
            self.terms[unique_term] = term_obj

        return term_obj

    def addCooccur(self, left_term, right_term):
        if right_term.id not in self.G[left_term.id]:
            self.G.add_edge(left_term.id, right_term.id, TF=0.)
        self.G[left_term.id][right_term.id]["TF"]+=1.
        
    def addOrUpdateComposedWord(self, cand):
        if cand.unique_kw not in self.candidates:
            self.candidates[cand.unique_kw] = cand
        else:
            self.candidates[cand.unique_kw].uptadeCand(cand)
        self.candidates[cand.unique_kw].tf += 1.

In [121]:
class composed_word(object):
    def __init__(self, terms): # [ (tag, word, term_obj) ]
        if terms == None:
             self.start_or_end_stopwords = True
             self.tags = set()
             return
        self.tags = set([''.join([ w[0] for w in terms ])])
        self.kw = ' '.join( [ w[1] for w in terms ] )
        self.unique_kw = self.kw.lower()
        self.size = len(terms)
        self.terms = [ w[2] for w in terms if w[2] != None ]
        self.tf = 0.
        self.integrity = 1.
        self.H = 1.
        self.start_or_end_stopwords = self.terms[0].stopword or self.terms[-1].stopword

    def uptadeCand(self, cand):
        for tag in cand.tags:
            self.tags.add( tag )

    def isValid(self):
        isValid = False
        for tag in self.tags:
            isValid = isValid or ( "u" not in tag and "d" not in tag )
        return isValid and not self.start_or_end_stopwords

    def get_composed_feature(self, feature_name, discart_stopword=True):
        list_of_features = [ getattr(term, feature_name) for term in self.terms if ( discart_stopword and not term.stopword ) or not discart_stopword ]
        sum_f  = sum(list_of_features)
        prod_f = np.prod(list_of_features)
        return ( sum_f, prod_f, prod_f /(sum_f + 1) )

    def build_features(self, doc_id=None, keys=None, rel=True, rel_approx=True, isVirtual=False, features=['WFreq', 'WRel', 'tf', 'WCase', 'WPos', 'WSpread'], _stopword=[True, False]):
        columns = []
        seen = set()
        features_cand = []

        if doc_id != None:
            columns.append('doc_id')
            features_cand.append(doc_id)

        if keys != None:
            if rel:
                columns.append('rel')
                if self.unique_kw in keys or isVirtual:
                    features_cand.append(1)
                    seen.add(self.unique_kw)
                else:
                    features_cand.append(0)

            if rel_approx:
                columns.append('rel_approx')
                max_gold_ = ('', 0.)
                for gold_key in keys:
                    dist = 1.-jellyfish.levenshtein_distance(gold_key, self.unique_kw ) / max(len(gold_key), len(self.unique_kw)) # _tL
                    if max_gold_[1] < dist:
                        max_gold_ = ( gold_key, dist )
                features_cand.append(max_gold_[1])

        columns.append('kw')
        features_cand.append(self.unique_kw)
        columns.append('h')
        features_cand.append(self.H)
        columns.append('tf')
        features_cand.append(self.tf)
        columns.append('size')
        features_cand.append(self.size)
        columns.append('isVirtual')
        features_cand.append(int(isVirtual))

        for feature_name in features:

            for discart_stopword in _stopword:
                (f_sum, f_prod, f_sum_prod) = self.get_composed_feature(feature_name, discart_stopword=discart_stopword)
                columns.append('%ss_sum_K%s' % ('n' if discart_stopword else '', feature_name) )
                features_cand.append(f_sum)

                columns.append('%ss_prod_K%s' % ('n' if discart_stopword else '', feature_name) )
                features_cand.append(f_prod)

                columns.append('%ss_sum_prod_K%s' % ('n' if discart_stopword else '', feature_name) )
                features_cand.append(f_sum_prod)

        return (features_cand, columns, seen)

    def updateH(self, features=None, isVirtual=False):
        sum_H  = 0.
        prod_H = 1.

        for (t, term_base) in enumerate(self.terms):
            if not term_base.stopword:
                sum_H += term_base.H
                prod_H *= term_base.H

            else:
                if STOPWORD_WEIGHT == 'bi':
                    prob_t1 = 0.
                    if term_base.G.has_edge(self.terms[t-1].id, self.terms[ t ].id):
                        prob_t1 = term_base.G[self.terms[t-1].id][self.terms[ t ].id]["TF"] / self.terms[t-1].tf

                    prob_t2 = 0.
                    if term_base.G.has_edge(self.terms[ t ].id, self.terms[t+1].id):
                        prob_t2 = term_base.G[self.terms[ t ].id][self.terms[t+1].id]["TF"] / self.terms[t+1].tf

                    prob = prob_t1 * prob_t2
                    prod_H *= (1 + (1 - prob ) )
                    sum_H -= (1 - prob)
                elif STOPWORD_WEIGHT == 'h':
                    sum_H += term_base.H
                    prod_H *= term_base.H
                elif STOPWORD_WEIGHT == 'none':
                    pass

        tf_used = 1.
        if features == None or "KPF" in features:
            tf_used = self.tf

        if isVirtual:
            tf_used = np.mean( [term_obj.tf for term_obj in self.terms] )

        self.H = prod_H / ( ( sum_H + 1 ) * tf_used )

    def updateH_old(self, features=None, isVirtual=False):
        sum_H  = 0.
        prod_H = 1.

        for (t, term_base) in enumerate(self.terms):
            if isVirtual and term_base.tf==0:
                continue

            if term_base.stopword:
                prob_t1 = 0.
                if term_base.G.has_edge(self.terms[t-1].id, self.terms[ t ].id):
                    prob_t1 = term_base.G[self.terms[t-1].id][self.terms[ t ].id]["TF"] / self.terms[t-1].tf

                prob_t2 = 0.
                if term_base.G.has_edge(self.terms[ t ].id, self.terms[t+1].id):
                    prob_t2 = term_base.G[self.terms[ t ].id][self.terms[t+1].id]["TF"] / self.terms[t+1].tf

                prob = prob_t1 * prob_t2
                prod_H *= (1 + (1 - prob ) )
                sum_H -= (1 - prob)
            else:
                sum_H += term_base.H
                prod_H *= term_base.H
        tf_used = 1.
        if features == None or "KPF" in features:
            tf_used = self.tf
        if isVirtual:
            tf_used = np.mean( [term_obj.tf for term_obj in self.terms] )
        self.H = prod_H / ( ( sum_H + 1 ) * tf_used )

In [122]:
class single_word(object):

    def __init__(self, unique, idx, graph):
        self.unique_term = unique
        self.id = idx
        self.tf = 0.
        self.WFreq = 0.0
        self.WCase = 0.0
        self.tf_a = 0.
        self.tf_n = 0.
        self.WRel = 1.0
        self.PL = 0.
        self.PR = 0.
        self.occurs = {}
        self.WPos = 1.0
        self.WSpread = 0.0
        self.H = 0.0
        self.stopword = False
        self.G = graph

        self.pagerank = 1.

    def updateH(self, maxTF, avgTF, stdTF, number_of_sentences, features=None):
        """if features == None or "WRel" in features:
            self.PL = self.WDL / maxTF
            self.PR = self.WDR / maxTF
            self.WRel = ( (0.5 + (self.PWL * (self.tf / maxTF) + self.PL)) + (0.5 + (self.PWR * (self.tf / maxTF) + self.PR)) )"""

        if features == None or "WRel" in features:
            self.PL = self.WDL / maxTF
            self.PR = self.WDR / maxTF
            self.WRel = ( (0.5 + (self.PWL * (self.tf / maxTF))) + (0.5 + (self.PWR * (self.tf / maxTF))) )

        if features == None or "WFreq" in features:
            self.WFreq = self.tf / (avgTF + stdTF)
        
        if features == None or "WSpread" in features:
            self.WSpread = len(self.occurs) / number_of_sentences
        
        if features == None or "WCase" in features:
            self.WCase = max(self.tf_a, self.tf_n) / (1. + math.log(self.tf))
        
        if features == None or "WPos" in features:
            self.WPos = math.log( math.log( 3. + np.median(list(self.occurs.keys())) ) )

        self.H = (self.WPos * self.WRel) / (self.WCase + (self.WFreq / self.WRel) + (self.WSpread / self.WRel))
        
    @property
    def WDR(self):
        return len( self.G.out_edges(self.id) )

    @property
    def WIR(self):
        return sum( [ d['TF'] for (u,v,d) in self.G.out_edges(self.id, data=True) ] )

    @property
    def PWR(self):
        wir = self.WIR
        if wir == 0:
            return 0
        return self.WDR / wir 
    
    @property
    def WDL(self):
        return len( self.G.in_edges(self.id) )

    @property
    def WIL(self):
        return sum( [ d['TF'] for (u,v,d) in self.G.in_edges(self.id, data=True) ] )

    @property
    def PWL(self):
        wil = self.WIL
        if wil == 0:
            return 0
        return self.WDL / wil 

    def addOccur(self, tag, sent_id, pos_sent, pos_text):
        if sent_id not in self.occurs:
            self.occurs[sent_id] = []

        self.occurs[sent_id].append( (pos_sent, pos_text) )
        self.tf += 1.

        if tag == "a":
            self.tf_a += 1.
        if tag == "n":
            self.tf_n += 1.

In [210]:
class TopicsExtractor(object):
    def __init__(self, top=20, features=None, stopwords=None):
        self.stopword_set = set(stopwords)
        self.n = 1
        self.top = top
        self.dedupLim = 0.9
        self.features = features
        self.windowsSize = 10
        self.dedu_function = self.jaro
#         fasttext_model = 'cc.en.300.bin'#'wiki/wiki.fr.bin'
#         self.model = load_model(fasttext_model)
        logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
        logging.info('model init')
    
    def txtPreprocessing(self, txt):
        _txtpp = []
        for s in txt:
            x = str(s).lower()
            x = re.sub(r'http\S+', ' ', x)
            x = re.sub('<.*?>', ' ', x)
            x = re.sub(' +', ' ', x)
            x = re.sub("\[.*?\]"," ",x)
            x = re.sub("['\"\\n]"," ", x)
            x = re.sub("www.\S+"," ", x)
            x = re.sub('[{’{»|«}~}]', ' ',x)
            x = re.sub('\s+', ' ', x)
            x = re.sub("(.)\\1{2,}", "\\1", x)
            x = re.sub(' +', ' ',x)
            x = x.strip()
            _txtpp.append(x)
        return _txtpp
    
    def txtEmbedding(self, txt):
        documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(txt)]
        doc2vec_args = {"vector_size": 300,
                "min_count": 1,
                "window": 15,
                "sample": 1e-5,
                "negative": 0,
                "hs": 1,
                "epochs": 50,
                "dm": 0,
                "dbow_words": 1,
                "workers":8,
               "documents":documents}
        model = Doc2Vec(**doc2vec_args)
        return model.dv.get_normed_vectors()


    
    def getClusers(self, embeddings):
        outliers = len(embeddings)*.22
        min_cluster_size, min_samples = 15, 2
        clusters = np.full(embeddings.shape[0], -1)
        _clusters = np.arange(len(clusters))

        umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', low_memory=False, verbose=False)
        if len(embeddings)//50000 < 2:
            _umap_embeddings = umap_model.fit_transform(embeddings)
            _embeddings = np.nan_to_num(_umap_embeddings) 
        else:
            umap_model.fit(embeddings[np.random.choice(embeddings.shape[0], 60000, replace=False), :])
            chunks = np.array_split(embeddings, len(embeddings)//50000)
            _umap_embeddings = []
            for chunk in chunks:
                _umap_embeddings.extend(umap_model.transform(chunk))
            _embeddings = np.nan_to_num(_umap_embeddings)    
        while(len(_clusters)>outliers):
            nbrCl = len(set(clusters))-1
            hdbscan_model =hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean',cluster_selection_method='eom', prediction_data=True)
            hdbscan_model.fit(_embeddings)
            indexes = np.argwhere(hdbscan_model.labels_!=-1)
            for i, v in zip(indexes, hdbscan_model.labels_[indexes]):
                clusters[_clusters[i]] = nbrCl + v
            _clusters = np.argwhere(hdbscan_model.labels_==-1)
            _embeddings = _embeddings[_clusters]
            if _embeddings.size == 0: break
            _embeddings = _embeddings.reshape(len(_clusters),-1)
            min_cluster_size, min_samples = 9, 1
        return clusters

    def getFreqKeywords(self,doc):
        tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=self.stopword_set)
        tfidf = tfidf_vectorizer.fit_transform([doc])
        score = np.argsort(np.asarray(tfidf.sum(axis=0)).ravel())[::-1]
        candidates = np.array(tfidf_vectorizer.get_feature_names_out())[score[:np.minimum(450,len(score)-1)]]
        return ' '.join(candidates)

    def jaro(self, cand1, cand2):
        return jellyfish.jaro_winkler(cand1, cand2 )

    def extract_keywords(self, text):
        try:
            if not(len(text) > 0):
                return []
            
            text = text.replace('\n\t',' ')
            dc = DataCore(text=text, stopword_set=self.stopword_set, windowsSize=self.windowsSize, n=self.n)
            dc.build_single_terms_features(features=self.features)
            dc.build_mult_terms_features(features=self.features)
            resultSet = []
            todedup = sorted([cc for cc in dc.candidates.values() if cc.isValid()], key=lambda c: c.H)

            if self.dedupLim >= 1.:
                return ([ (cand.H, cand.unique_kw) for cand in todedup])[:self.top]

            for cand in todedup:
                toadd = True
                for (h, candResult) in resultSet:
                    dist = self.dedu_function(cand.unique_kw, candResult.unique_kw)
                    if dist > self.dedupLim:
                        toadd = False
                        break
                if toadd:
                    resultSet.append( (cand.H, cand) )
                if len(resultSet) == self.top:
                    break

            return " ".join([cand.kw for (h,cand) in resultSet])
        except Exception as e:
            print(f"Warning! Exception: {e} generated by the following text: '{text}' ")
            return []
    
    def groupBy(self, txtpp, clusters, nbrClusters):
        docs = np.full((nbrClusters, ), "",dtype=object)
        for cluster, txt in zip(clusters,txtpp):
            docs[cluster+1] += txt
        return docs
    
    def extract_topics_keywords(self, docs):
        topics = {}
        for i, doc in enumerate(docs):
            topics[i-1]=self.extract_keywords(doc)
        return topics

    def clusterreduction(self, docs, nbrClusters, clusters):
        freqKey = []
        embeddings = []
        for doc in docs[1:]: freqKey.append(self.getFreqKeywords(doc))
        for doc in freqKey:embeddings.append(self.model.get_sentence_vector(doc))
        hdbscan_model =hdbscan.HDBSCAN(min_cluster_size=3, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
        hdbscan_model.fit(embeddings)
        print(hdbscan_model.labels_, set(hdbscan_model.labels_))
        mapper = []
        for i, lable in enumerate(hdbscan_model.labels_):
            if lable !=-1:
                mapper.append(nbrClusters + lable)
            else:
                mapper.append(i + 1)
        _args = np.array(sorted(mapper))
        
        for i, lable in enumerate(mapper): 
            mapper[i]=np.argwhere(lable == _args)[0].item()
        topicMapper = {}
        
        for o,n in zip(docs[1:],mapper): 
            topicMapper[o] = n 
        for i, v in enumerate(clusters):
            try:
                clusters[i]= topicMapper[v]
            except: pass
        return clusters
    
    def extract_topics(self, text):
        txtpp = self.txtPreprocessing(text)
        logging.info('end of text pre processing')
        embeddings = self.txtEmbedding(txtpp)
        logging.info('end of text embedding')
        clusters = self.getClusers(embeddings)
        nbrClusters = len(set(clusters))
        logging.info(f'end of clustring : {nbrClusters}')
        docs = self.groupBy(txtpp, clusters, nbrClusters)
        topics = self.extract_topics_keywords(docs)
        logging.info('end of key extra')
        return topics, clusters

In [278]:
data = pd.read_csv('/data/notebooks/others/french_tweets.csv')

In [268]:
data = pd.read_parquet('/data/big-data-collection/OTHERS/data_esg.parquet')
data = data[['TITRE','CONTENU']]
data["txt"] = data['TITRE'] +" "+data['CONTENU']

In [10]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
text = newsgroups_train.data

In [126]:
stopWords = open('./stopwordEng', 'r').readlines()
for i in range(len(stopWords)):
    stopWords[i] = stopWords[i].replace('\n', '').replace(' ', '')
stopWords += list(set(stopWords))

In [211]:
topicModel = TopicsExtractor(top=30, stopwords=stopWords)
out = topicModel.extract_topics(text)

2022-07-28 14:47:03,025 - model init
2022-07-28 14:47:06,725 - end of text pre processing
2022-07-28 14:47:06,740 - collecting all words and their counts
2022-07-28 14:47:06,741 - Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2022-07-28 14:47:06,741 - PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-07-28 14:47:07,534 - PROGRESS: at example #10000, processed 11130724 words (14055163 words/s), 76 word types, 0 tags
2022-07-28 14:47:08,199 - collected 89 word types and 18846 unique tags from a corpus of 18846 examples and 20553241 words
2022-07-28 14:47:08,200 - Creating a fresh vocabulary
2022-07-28 14:47:08,201 - Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 89 unique words (100.00% of original 89, drops 0)', 'datetime': '2022-07-28T14:47:08.201051', 'gensim': '4.2.0', 'python': '3.8.0 (default, Dec  9 2021, 17:53:27) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.0-1080-azu

In [212]:
out[0]

{-1: 'people time god government file space program windows armenian data law key question read drive list power day car set disk jesus send hard university email support word called public',
 0: 'max bhj giz scx bxn okz air rlk fij fyn qax ahf kjz ghj gcx nrhj biz wwiz uww qtm mcx mtm sqs khf syx nuy vmk ajz xte rck',
 1: 'people homosexual god true evidence belief time theism moral claim bible reason gay read argument lot sexual statement jesus question children force wrong issue support church person subject university agree',
 2: 'hitler dollars nazis government chancellor time party avs games parties people german mission motif power support constitution coalition esd conductive increase manager reichstag democratic hindenburg national lost software jesus drive',
 3: 'file image program jpeg people dos version time data software pub graphics server ftp format window output display subject stephanopoulos set president entry source package space support color code gif',
 4: 'cosy pa

In [134]:
def tpp(txt):
    x = str(txt).lower()
    x = re.sub(r'http\S+', '', x)
    x = re.sub(r'@[^\s]+', '', x)
    x = re.sub(r'#[^\s]+', '',x)
    x = re.sub('<.*?>', ' ', x)
    x = re.sub(' +', ' ', x)
    x = re.sub("\[.*?\]"," ",x)
    x = re.sub("[()!?',:;!.\"\\n]"," ", x)
    x = re.sub("['\"\\n]"," ", x)
    x = re.sub("www.\S+"," ", x)
    x = re.sub('[{!"#$%&\'()*’+,-./:;<=>?@[\\]^_`{»|«}~}]', ' ',x)
    x = re.sub('\s+', ' ', x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    x = re.sub(' +', ' ',x)
    x = x.strip()
    x = " ".join([word for word in x.split(" ") if word not in stopWords])
    return x

In [185]:
txtClean = []
for d in text:
    txtClean.append(tpp(d))

In [186]:
txtClean[5]

'school lab assistant bunch experimental psychologists bell labs visual perception memory experiments vector type displays 1 millisecond refresh rates common 1 200th practical experimenters 5 milliseconds 4 6 steve'

In [138]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
# data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(txtClean))

['bashers', 'pens', 'fans', 'pretty', 'confused', 'lack', 'posts', 'pens', 'massacre', 'devils', 'bit', 'puzzled', 'bit', 'relieved', 'pittsburghers', 'relief', 'bit', 'praise', 'pens', 'killing', 'devils', 'worse', 'jagr', 'regular', 'season', 'stats', 'lot', 'fun', 'watch', 'playoffs']


In [149]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])


2022-07-28 13:14:24,819 - adding document #0 to Dictionary<0 unique tokens: []>
2022-07-28 13:14:25,471 - adding document #10000 to Dictionary<61185 unique tokens: ['bashers', 'beat', 'bit', 'bowman', 'confused']...>
2022-07-28 13:14:26,041 - built Dictionary<84565 unique tokens: ['bashers', 'beat', 'bit', 'bowman', 'confused']...> from 18846 documents (total 1359697 corpus positions)
2022-07-28 13:14:26,042 - Dictionary lifecycle event {'msg': "built Dictionary<84565 unique tokens: ['bashers', 'beat', 'bit', 'bowman', 'confused']...> from 18846 documents (total 1359697 corpus positions)", 'datetime': '2022-07-28T13:14:26.042187', 'gensim': '4.2.0', 'python': '3.8.0 (default, Dec  9 2021, 17:53:27) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.0-1080-azure-x86_64-with-glibc2.27', 'event': 'created'}


[[(0, 1), (1, 1), (2, 3), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 5), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1)]]


In [168]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v',processes=8)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_ld)

2022-07-28 13:49:24,417 - using ParallelWordOccurrenceAccumulator<processes=8, batch_size=64> to estimate probabilities from sliding windows
2022-07-28 13:49:24,763 - worker encountered unexpected exception
Traceback (most recent call last):
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 561, in run
    self._run()
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 581, in _run
    self.accumulator.partial_accumulate(docs, self.window_size)
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 353, in partial_accumulate
    super(WordOccurrenceAccumulator, self).accumulate(texts, window_size)
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 296, in accumulate
    self.analyze_text(virtual_document, doc_num)
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 360, in analyze_tex

KeyboardInterrupt: 

In [ ]:
"fij kf".split()

In [218]:
topicc = []
for i in out[0].keys():
    topicc.append(out[0][i].split())

In [188]:
topicc

[['file',
  'people',
  'dos',
  'time',
  'god',
  'president',
  'space',
  'stephanopoulos',
  'program',
  'version',
  'jpeg',
  'health',
  'data',
  'output',
  'armenian',
  'software',
  'ftp',
  'jesus',
  'windows',
  'questions',
  'children',
  'image',
  'united',
  'adl',
  'national',
  'april',
  'set',
  'public',
  'control',
  'day'],
 ['dollars',
  'motif',
  'avs',
  'games',
  'mission',
  'time',
  'constitution',
  'esd',
  'conductive',
  'increase',
  'government',
  'manager',
  'ms-w',
  'software',
  'jesus',
  'graphics',
  'power',
  'disks',
  'floppy',
  'drive',
  'unleaded',
  'card',
  'spent',
  'question',
  'phill',
  'program',
  'forget',
  'national',
  'features',
  'network'],
 ['people',
  'internet',
  'zionism',
  'ethernet',
  'christianity',
  'node',
  'decnet',
  'address',
  'worth',
  'homosexual',
  'religious',
  'chancellor',
  'voted',
  'hitler',
  'space',
  'count',
  'connected',
  'vaxstation',
  'live',
  'statement',
  'a

In [166]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

2022-07-28 13:46:04,865 - using symmetric alpha at 0.1
2022-07-28 13:46:04,866 - using symmetric eta at 0.1
2022-07-28 13:46:04,875 - using serial LDA version on this node
2022-07-28 13:46:04,950 - running online LDA training, 10 topics, 10 passes over the supplied corpus of 18846 documents, updating every 1500 documents, evaluating every ~15000 documents, iterating 50x with a convergence threshold of 0.001000
2022-07-28 13:46:04,951 - training LDA model using 15 processes
2022-07-28 13:46:05,521 - PROGRESS: pass 0, dispatched chunk #0 = documents up to #100/18846, outstanding queue size 1
2022-07-28 13:46:05,533 - PROGRESS: pass 0, dispatched chunk #1 = documents up to #200/18846, outstanding queue size 2
2022-07-28 13:46:05,534 - PROGRESS: pass 0, dispatched chunk #2 = documents up to #300/18846, outstanding queue size 3
2022-07-28 13:46:05,534 - PROGRESS: pass 0, dispatched chunk #3 = documents up to #400/18846, outstanding queue size 4
2022-07-28 13:46:05,535 - PROGRESS: pass 0, di

In [ ]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# Preprocess documents
cleaned_docs = topic_model._preprocess_text(docs)

# Extract vectorizer and tokenizer from BERTopic
vectorizer = topic_model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [tokenizer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

In [214]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1), stop_words="english")
vectorizer.fit(txtClean)

CountVectorizer(stop_words='english')

In [215]:
words = vectorizer.get_feature_names()

/data/env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [216]:
tokenizer = vectorizer.build_tokenizer()
tokens = [tokenizer(doc) for doc in txtClean]

In [217]:
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]

2022-07-28 14:54:19,924 - adding document #0 to Dictionary<0 unique tokens: []>
2022-07-28 14:54:20,680 - adding document #10000 to Dictionary<76191 unique tokens: ['bashers', 'beat', 'bit', 'bowman', 'confused']...>
2022-07-28 14:54:21,336 - built Dictionary<109445 unique tokens: ['bashers', 'beat', 'bit', 'bowman', 'confused']...> from 18846 documents (total 1478445 corpus positions)
2022-07-28 14:54:21,337 - Dictionary lifecycle event {'msg': "built Dictionary<109445 unique tokens: ['bashers', 'beat', 'bit', 'bowman', 'confused']...> from 18846 documents (total 1478445 corpus positions)", 'datetime': '2022-07-28T14:54:21.337357', 'gensim': '4.2.0', 'python': '3.8.0 (default, Dec  9 2021, 17:53:27) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.0-1080-azure-x86_64-with-glibc2.27', 'event': 'created'}


In [ ]:
coherence_model = CoherenceModel(topics=topicc, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

2022-07-28 14:54:38,860 - using ParallelWordOccurrenceAccumulator<processes=15, batch_size=64> to estimate probabilities from sliding windows
2022-07-28 14:54:40,331 - worker encountered unexpected exception
Traceback (most recent call last):
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 561, in run
    self._run()
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 581, in _run
    self.accumulator.partial_accumulate(docs, self.window_size)
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 353, in partial_accumulate
    super(WordOccurrenceAccumulator, self).accumulate(texts, window_size)
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 296, in accumulate
    self.analyze_text(virtual_document, doc_num)
  File "/data/env/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 360, in analyze_te

In [190]:
coherence

-5.579879075280098